# Incidence Angle model ensemble
Need to discard the training data that does not include an incidence angle

In [1]:
# load data
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from keras.models import Model
from keras.layers import Flatten, Dense, Input
from keras.layers import Convolution2D, MaxPooling2D, BatchNormalization, Dropout
from keras.layers import GlobalMaxPooling2D, concatenate
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.utils import np_utils
from keras import regularizers
import random

# get list of hold out data for ensemble training
random.seed(17332)
nums = random.sample(range(0,1507),60)


training_path = "../../Test_data/train.json"
testing_path = "../../Test_data/test.json"

train_data_whole = pd.read_json(training_path)
train_data = train_data_whole.select(lambda x: x not in nums)

test_data = pd.read_json(testing_path)


# no third for inc angle
def get_scaled_imgs(df):
    imgs = []
    
    for i, row in df.iterrows():
        #make 75x75 image
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        # band_3 = band_1 + band_2 # plus since log(x*y) = log(x) + log(y)
        
        # Rescale
        # a = (band_1 - band_1.mean()) / (band_1.max() - band_1.min())
        # b = (band_2 - band_2.mean()) / (band_2.max() - band_2.min())
        # c = (band_3 - band_3.mean()) / (band_3.max() - band_3.min())

        imgs.append(np.dstack((band_1, band_2)))    # , c)))

    return np.array(imgs)

X = get_scaled_imgs(train_data)

# get only data that has a non 0 inc angle
train_data.inc_angle = train_data.inc_angle.replace('na',0)
idx_tr = np.where(train_data.inc_angle>0)

Y = train_data["is_iceberg"]
inc_angle = train_data["inc_angle"]

Y_inc = Y.iloc[idx_tr[0]]
X_inc = X[idx_tr[0],...]
inc_angle_inc = inc_angle.iloc[idx_tr[0]]
# inc_angle_norm = (inc_angle_inc - inc_angle_inc.mean()) / (inc_angle_inc.max() - inc_angle_inc.min())


X_full = X_inc
Y_full = Y_inc
inc_full = inc_angle_inc

X_train, X_test, y_train, y_test, inc_train, inc_test = train_test_split(X_inc, Y_inc,
                                                    inc_angle_inc, test_size = 0.25, 
                                                                         random_state=42)

print("size x_train", len(X_train))
print("size x_test", len(X_test))
print("size inc train", len(inc_train))
print("size inc test", len(inc_test))
print("size y_train", len(y_train))
print("size y_test", len(y_test))

print("size x full", len(X_full))
print("size y full", len(Y_full))
print("size inc full", len(inc_full))

#print(X_full)
#print(Y_full)
#print(inc_full)



Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:25: FutureWarning: 'select' is deprecated and will be removed in a future release. You can use .loc[labels.map(crit)] as a replacement


size x_train 1058
size x_test 353
size inc train 1058
size inc test 353
size y_train 1058
size y_test 353
size x full 1411
size y full 1411
size inc full 1411
0       0
1       0
2       1
3       0
4       0
5       1
6       1
7       0
8       0
9       0
10      1
11      0
12      1
13      1
15      0
16      0
17      0
18      0
19      1
20      0
21      1
22      0
23      1
24      0
25      1
26      1
27      0
28      1
30      0
31      0
       ..
1479    1
1480    0
1481    1
1482    0
1483    0
1484    1
1485    0
1486    1
1487    1
1488    0
1489    1
1490    1
1491    0
1492    1
1493    1
1494    0
1495    0
1496    1
1497    1
1498    1
1499    0
1500    1
1501    1
1502    1
1503    0
1504    1
1505    0
1506    1
1507    0
1508    1
Name: is_iceberg, Length: 1411, dtype: int64


In [2]:
# create the model
img_input = Input(shape=(75,75,2))
ang_input = Input(shape=(1,))
x = Convolution2D(64, (3,3), activation='relu', padding='same',
                  kernel_regularizer=regularizers.l2(0.1),
                  bias_regularizer=regularizers.l2(0.1), name='block1_conv1')(img_input)
x = Convolution2D(64, (3,3), activation='relu', padding='same',
                  kernel_regularizer=regularizers.l2(0.1),
                  bias_regularizer=regularizers.l2(0.1), name='block1_conv2')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)
x = Dropout(0.2)(x)

x = Convolution2D(128, (3,3), activation='relu', padding='same',
                  kernel_regularizer=regularizers.l2(0.1),
                  bias_regularizer=regularizers.l2(0.1), name='block2_conv1')(x)
x = Convolution2D(128, (3,3), activation='relu', padding='same',
                  kernel_regularizer=regularizers.l2(0.1),
                  bias_regularizer=regularizers.l2(0.1), name='block2_conv2')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)
x = Dropout(0.2)(x)

x = Convolution2D(256, (3,3), activation='relu', padding='same',
                  kernel_regularizer=regularizers.l2(0.1),
                  bias_regularizer=regularizers.l2(0.1), name='block3_conv1')(x)
x = Convolution2D(256, (3,3), activation='relu', padding='same',
                  kernel_regularizer=regularizers.l2(0.1),
                  bias_regularizer=regularizers.l2(0.1), name='block3_conv2')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)
x = Dropout(0.2)(x)

x = Flatten(name='flatten')(x)
x = concatenate([x, ang_input])
x = Dense(256, kernel_regularizer=regularizers.l2(0.1),
          bias_regularizer=regularizers.l2(0.1))(x)
x = Dropout(0.5)(x)
x = Dense(128, kernel_regularizer=regularizers.l2(0.1),
          bias_regularizer=regularizers.l2(0.1))(x)
x = Dropout(0.5)(x)
main_output = Dense(1, activation='sigmoid', name='predictions')(x)

model = Model(inputs=[img_input,ang_input], outputs= [main_output], name='vgg_inc')
model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.00001), metrics=['accuracy'])
model.summary()


def get_callbacks(filepath, patience=2):
    es = EarlyStopping('val_loss', patience=patience, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [es, msave]
file_path = "model_weights_inc_angle_TWO_reg1_ensem.hdf5"
callbacks = get_callbacks(filepath=file_path, patience=15)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 75, 75, 2)    0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 75, 75, 64)   1216        input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 75, 75, 64)   36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 37, 37, 64)   0           block1_conv2[0][0]               
__________________________________________________________________________________________________
dropout_1 

# Notes
0.5 on the reg for all got to about 90 with 256 and 128 on the fc layers

In [3]:
# fitting
model.fit([X_full, inc_full], Y_full, batch_size=32, 
          epochs=240, verbose=1) #, callbacks=callbacks, 
          # validation_data = ([X_test, inc_test], y_test))

Epoch 1/240
1411/1411 [==============================] - 8s 6ms/step - loss: 138.4879 - acc: 0.5245
Epoch 2/240
1411/1411 [==============================] - 5s 4ms/step - loss: 135.6230 - acc: 0.5195
Epoch 3/240
1411/1411 [==============================] - 5s 4ms/step - loss: 133.0124 - acc: 0.5067
Epoch 4/240
1411/1411 [==============================] - 5s 4ms/step - loss: 130.3574 - acc: 0.5223
Epoch 5/240
1411/1411 [==============================] - 5s 4ms/step - loss: 127.8026 - acc: 0.5202
Epoch 6/240
1411/1411 [==============================] - 5s 4ms/step - loss: 125.2474 - acc: 0.5124
Epoch 7/240
1411/1411 [==============================] - 5s 4ms/step - loss: 122.7213 - acc: 0.5393
Epoch 8/240
1411/1411 [==============================] - 5s 4ms/step - loss: 120.2235 - acc: 0.5556
Epoch 9/240
1411/1411 [==============================] - 5s 4ms/step - loss: 117.7962 - acc: 0.5400
Epoch 10/240
1411/1411 [==============================] - 5s 4ms/step - loss: 115.3335 - acc: 0.5641

1411/1411 [==============================] - 5s 4ms/step - loss: 22.1465 - acc: 0.7845
Epoch 83/240
1411/1411 [==============================] - 5s 4ms/step - loss: 21.6241 - acc: 0.8072
Epoch 84/240
1411/1411 [==============================] - 5s 4ms/step - loss: 21.1232 - acc: 0.8094
Epoch 85/240
1411/1411 [==============================] - 5s 4ms/step - loss: 20.6360 - acc: 0.8051
Epoch 86/240
1411/1411 [==============================] - 5s 4ms/step - loss: 20.1646 - acc: 0.8044
Epoch 87/240
1411/1411 [==============================] - 5s 4ms/step - loss: 19.6988 - acc: 0.7987
Epoch 88/240
1411/1411 [==============================] - 5s 4ms/step - loss: 19.2493 - acc: 0.7994
Epoch 89/240
1411/1411 [==============================] - 5s 4ms/step - loss: 18.8054 - acc: 0.8009
Epoch 90/240
1411/1411 [==============================] - 5s 4ms/step - loss: 18.3764 - acc: 0.8094
Epoch 91/240
1411/1411 [==============================] - 5s 4ms/step - loss: 17.9677 - acc: 0.8058
Epoch 92/240


1411/1411 [==============================] - 5s 4ms/step - loss: 4.4015 - acc: 0.8533
Epoch 164/240
1411/1411 [==============================] - 5s 4ms/step - loss: 4.3382 - acc: 0.8462
Epoch 165/240
1411/1411 [==============================] - 5s 4ms/step - loss: 4.2674 - acc: 0.8498
Epoch 166/240
1411/1411 [==============================] - 5s 4ms/step - loss: 4.1991 - acc: 0.8512
Epoch 167/240
1411/1411 [==============================] - 5s 4ms/step - loss: 4.1360 - acc: 0.8498
Epoch 168/240
1411/1411 [==============================] - 5s 4ms/step - loss: 4.0677 - acc: 0.8526
Epoch 169/240
1411/1411 [==============================] - 5s 4ms/step - loss: 4.0164 - acc: 0.8490
Epoch 170/240
1411/1411 [==============================] - 5s 4ms/step - loss: 3.9578 - acc: 0.8512
Epoch 171/240
1411/1411 [==============================] - 5s 4ms/step - loss: 3.8974 - acc: 0.8554
Epoch 172/240
1411/1411 [==============================] - 5s 4ms/step - loss: 3.8351 - acc: 0.8498
Epoch 173/240


In [ ]:
# if doing no val set (i.e. final) only
filepath_full = 'model_weights_inc2_full_reg01_ensem.hdf5'
model.save(filepath_full)

In [ ]:
# from above, need to see the amount of epochs (80 +) that val stops increasing at, then do 
# that in one full go and save the model

from keras.models import load_model
import pandas as pd
# if not using val
file_path_use = filepath_full
# if using val
#file_path_use = file_path
inf_model = load_model(file_path_use)

score = inf_model.evaluate([X_full,inc_full], Y_full, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# for hold out submission to train ensemble
hold_out_train = train_data.iloc[nums]


X_sub = get_scaled_imgs(hold_out_train)
inc_angle_t = hold_out_train["inc_angle"]
# inc_angle_norm_t = (inc_angle_t - inc_angle_t.mean()) / (inc_angle_t.max() - inc_angle_t.min())


predicted_test=inf_model.predict([X_sub, inc_angle_t])

print("len of pred test", len(predicted_test))
print("len of id", len(hold_out_train['id']))

submission = pd.DataFrame()
submission['id']=hold_out_train['id']
submission['is_iceberg']=predicted_test
submission.to_csv('sub_full_inc2_ensem.csv', index=False)